# Lab 03.1 — Datos Sintéticos con Synthea

**BE3006 · Análisis de Datos Biomédicos · Ciclo 1, 2026**

---

En **Lab 1** construiste un EHR relacional.  
En **Lab 2** normalizaste su semántica con ICD-10 y LOINC.  
En **Lab 3** convertiste esos datos a FHIR y ensamblaste Bundles de intercambio.

Ahora la pregunta natural:

> ¿Cómo se generan miles de registros clínicos realistas — con terminologías correctas, trayectorias de enfermedad coherentes y distribuciones epidemiológicas reales — para investigación, pruebas y entrenamiento de modelos?

La respuesta: **Synthea**, un simulador de poblaciones de pacientes sintéticos open-source.

---

```
┌────────────────────┐     ┌──────────────────────┐     ┌─────────────────────┐
│  Módulos de        │     │  Motor Synthea        │     │  Salida por         │
│  enfermedad        │────▶│  (simulación          │────▶│  paciente:          │
│  (JSON / máquinas  │     │   epidemiológica      │     │  CSV + FHIR R4      │
│   de estado)       │     │   probabilística)     │     │  Bundle             │
└────────────────────┘     └──────────────────────┘     └─────────────────────┘
```

---

**Objetivos:**

1. Entender qué son los datos sintéticos y por qué importan en salud digital
2. Explorar la estructura de salida de Synthea: archivos CSV y Bundles FHIR R4
3. Analizar una población sintética: demografía y epidemiología básica
4. Comparar un Bundle FHIR generado por Synthea con el que construiste en Lab 3
5. Reflexionar sobre los usos y limitaciones de datos sintéticos en biomedicina


---

## Parte 0 — Setup

Asegúrate de que Docker esté corriendo: `docker compose up -d`  
Luego abre Jupyter en `http://localhost:8888` y navega a `work/notebooks/`.


In [ ]:
import json
import warnings
from datetime import date
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from fhir.resources.bundle import Bundle

# ─── Paths ────────────────────────────────────────────────────────────────────
DATA_DIR = Path('../data')
FHIR_DIR = Path('../fhir')

# ─── Display settings ─────────────────────────────────────────────────────────
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 80)
sns.set_theme(style='whitegrid', palette='muted')
warnings.filterwarnings('ignore')

# ─── Verificar archivos de datos ──────────────────────────────────────────────
csv_files = ['patients.csv', 'conditions.csv', 'encounters.csv',
             'medications.csv', 'observations.csv']
missing_csv = [f for f in csv_files if not (DATA_DIR / f).exists()]
fhir_files  = sorted(FHIR_DIR.glob('*.json'))

if missing_csv:
    print('⚠️  Faltan archivos CSV en data/:')
    for f in missing_csv:
        print(f'   • {f}')
    print()
    print('Para generarlos, ejecuta desde el directorio de Synthea:')
    print('  ./run_synthea -s 42 -p 100 \\')
    print('    --exporter.csv.export=true \\')
    print('    --exporter.fhir.export=true \\')
    print('    Massachusetts')
    print()
    print('Luego copia los CSV de synthea/output/csv/ → labs/lab-3.1-synthetic-data/data/')
    print('  y los JSON de synthea/output/fhir/ → labs/lab-3.1-synthetic-data/fhir/')
else:
    print('✓ Archivos CSV encontrados')
    if fhir_files:
        print(f'✓ {len(fhir_files)} archivos FHIR encontrados')
    else:
        print('⚠️  No hay archivos FHIR en fhir/ (Parte 5 los requiere)')
    print('\nSetup completo ✓')


---

## Parte 1 — ¿Qué es Synthea y por qué importan los datos sintéticos?

### El problema: privacidad vs. acceso a datos

Los datos clínicos reales son invaluables para investigación, pero:

- **Están protegidos por ley** (HIPAA en EE.UU., GDPR en Europa, legislaciones locales).
- **Compartirlos requiere consentimiento** del paciente y procesos de anonimización complejos.
- **La anonimización nunca es perfecta**: con suficiente contexto, un paciente puede re-identificarse.

Esto genera un dilema: los modelos de IA, los sistemas de EHR y los protocolos de investigación necesitan datos para desarrollarse y probarse, pero los datos reales son difíciles de obtener.

### La solución: datos sintéticos

Los datos sintéticos son datos **generados artificialmente** que:
- No corresponden a ningún individuo real
- Preservan las **distribuciones estadísticas** de la población real
- Incluyen **terminologías clínicas** correctas (SNOMED-CT, LOINC, RxNorm)
- Pueden generarse en la cantidad y con las características que se necesiten

### Usos principales en biomedicina:

| Uso | Ejemplo |
|-----|--------|
| Pruebas de sistemas | Probar un EHR nuevo sin datos reales |
| Entrenamiento de modelos IA | Entrenar un clasificador de diagnósticos |
| Investigación epidemiológica | Estudiar prevalencias sin acceder a datos hospitalarios |
| Educación | Laboratorios como este |
| Benchmarks | Comparar algoritmos de análisis clínico |


### ¿Cómo funciona Synthea?

Synthea modela la vida completa de un paciente usando **máquinas de estado probabilísticas**.
Cada enfermedad tiene su propio módulo (un archivo JSON) con:

- Estados (`Initial`, `Onset`, `Diagnosed`, `Treated`, `Death`, etc.)
- Transiciones basadas en probabilidades epidemiológicas reales
- Códigos terminológicos correctos (SNOMED-CT para condiciones, LOINC para labs, RxNorm para medicamentos)

```
  Initial
    │
    ▼  (probabilidad basada en edad, sexo, etnicidad)
  Assign_Hypertension_Probability
    │
    ├──[no HTN]──▶  Terminal
    │
    └──[HTN]──▶  Onset_Hypertension
                    │
                    ▼
                 Diagnose_Hypertension  ←── SNOMED: 59621000
                    │
                    ▼
                 Hypertension_Followup_Encounter
                    │
                    ├──[no control]──▶  Add_Medication  ←── RxNorm
                    │
                    └──[controlled]──▶  Terminal
```

Synthea tiene módulos para más de 60 enfermedades: diabetes, EPOC, insuficiencia cardiaca, cáncer de mama, opioides, COVID-19, y más.


### Los datos de este lab

Los archivos en `data/` y `fhir/` fueron generados con el siguiente comando:

```bash
# Desde el directorio de Synthea
./run_synthea -s 42 -p 100 \
  --exporter.csv.export=true \
  --exporter.fhir.export=true \
  Massachusetts
```

Parámetros:
- `-s 42` → semilla aleatoria (reproducibilidad: con la misma semilla obtienes los mismos pacientes)
- `-p 100` → 100 pacientes
- `--exporter.csv.export=true` → genera CSVs relacionales
- `--exporter.fhir.export=true` → genera un Bundle FHIR R4 por paciente
- `Massachusetts` → estado de EE.UU. (define demografía de base)

> 💡 Con la misma semilla (`-s 42`) y el mismo número de pacientes (`-p 100`), Synthea siempre genera exactamente los mismos datos. Esto hace el análisis **reproducible**.


#### Ejercicio 1.1 — Reflexión sobre datos sintéticos

Basándote en la descripción de Synthea y sus usos, responde:

**a)** ¿Por qué es importante que los datos sintéticos usen terminologías clínicas reales (SNOMED-CT, LOINC, RxNorm) en lugar de códigos inventados?

**b)** ¿Cuál es una limitación fundamental de los datos sintéticos que los hace inadecuados para ciertos tipos de investigación clínica?

**c)** ¿Qué ventaja tiene generar datos con una semilla fija (`-s 42`) en un contexto de investigación?


**Tus respuestas:**

a) ...

b) ...

c) ...


---

## Parte 2 — Estructura de datos CSV de Synthea

Synthea puede exportar datos en formato CSV relacional, con una tabla por tipo de registro clínico.  
Esta estructura debería resultarte familiar: es muy similar al EHR que construiste en Lab 1.


In [ ]:
# Cargar todos los archivos CSV de Synthea
patients     = pd.read_csv(DATA_DIR / 'patients.csv',
                           parse_dates=['BIRTHDATE', 'DEATHDATE'])
conditions   = pd.read_csv(DATA_DIR / 'conditions.csv',
                           parse_dates=['START', 'STOP'])
encounters   = pd.read_csv(DATA_DIR / 'encounters.csv',
                           parse_dates=['START', 'STOP'])
medications  = pd.read_csv(DATA_DIR / 'medications.csv',
                           parse_dates=['START', 'STOP'])
observations = pd.read_csv(DATA_DIR / 'observations.csv',
                           parse_dates=['DATE'])

print('Registros cargados:')
print(f'  patients:     {len(patients):>6,}')
print(f'  conditions:   {len(conditions):>6,}')
print(f'  encounters:   {len(encounters):>6,}')
print(f'  medications:  {len(medications):>6,}')
print(f'  observations: {len(observations):>6,}')


### 2.1 `patients.csv` — Demografía

Cada fila representa un paciente único. Las columnas clave son:

| Columna | Descripción |
|---------|-------------|
| `Id` | UUID único del paciente (actúa como `subject_id` en Lab 1) |
| `BIRTHDATE` / `DEATHDATE` | Fechas de nacimiento y fallecimiento |
| `GENDER` | M / F |
| `RACE` / `ETHNICITY` | Clasificación demográfica |
| `CITY`, `STATE` | Ubicación geográfica |
| `HEALTHCARE_EXPENSES` | Costo total de atención médica en vida |


In [ ]:
display(patients[['Id', 'BIRTHDATE', 'DEATHDATE', 'GENDER', 'RACE',
                  'ETHNICITY', 'CITY', 'HEALTHCARE_EXPENSES']].head(5))


### 2.2 `conditions.csv` — Diagnósticos

Cada fila representa una condición clínica de un paciente. Observa la columna `CODE`:

| Columna | Descripción |
|---------|-------------|
| `PATIENT` | UUID del paciente (clave foránea → `patients.Id`) |
| `ENCOUNTER` | UUID del encuentro en que fue diagnosticado |
| `CODE` | Código **SNOMED-CT** de la condición |
| `DESCRIPTION` | Descripción legible de la condición |
| `START` / `STOP` | Inicio y resolución de la condición |

> 🔗 **Conexión con Lab 2**: Synthea usa **SNOMED-CT** para todas las condiciones, no ICD-10.
> Ambos son sistemas de terminología válidos; la diferencia es el nivel de granularidad.
> SNOMED-CT es más clínico y detallado; ICD-10 se usa más para facturación y epidemiología.


In [ ]:
display(conditions[['PATIENT', 'START', 'STOP', 'CODE', 'DESCRIPTION']].head(8))
print(f'\nCondiciones únicas en el dataset: {conditions["DESCRIPTION"].nunique()}')


### 2.3 `encounters.csv` — Visitas clínicas

Cada fila es un encuentro clínico (visita, hospitalización, emergencia).

| Columna | Descripción |
|---------|-------------|
| `PATIENT` | UUID del paciente |
| `ENCOUNTERCLASS` | Tipo: `ambulatory`, `inpatient`, `emergency`, `wellness`, etc. |
| `CODE` | Código SNOMED-CT del tipo de encuentro |
| `REASONCODE` / `REASONDESCRIPTION` | Motivo del encuentro |
| `TOTAL_CLAIM_COST` | Costo total de la visita |

> 🔗 **Conexión con Lab 1**: Esta tabla equivale a `admissions` en el EHR que construiste.
> La diferencia clave: Synthea genera múltiples encuentros por paciente a lo largo de su vida.


In [ ]:
display(encounters[['PATIENT', 'START', 'STOP', 'ENCOUNTERCLASS',
                    'DESCRIPTION', 'TOTAL_CLAIM_COST']].head(5))

print('\nDistribución por tipo de encuentro:')
display(encounters['ENCOUNTERCLASS'].value_counts().to_frame())


#### Ejercicio 2.1 — Comparación con el EHR del Lab 1

El EHR que construiste en Lab 1 tenía estas tablas: `patients`, `admissions`, `diagnoses`, `labevents`.

**a)** Mapea cada tabla de Synthea con su equivalente más cercano en el EHR de Lab 1.  
   ¿Qué tabla de Synthea no tiene equivalente directo en el Lab 1?

**b)** ¿Por qué `observations.csv` de Synthea usa **LOINC** para los códigos, y no SNOMED-CT?  
   _(Pista: recuerda qué tipo de datos cubre LOINC según Lab 2)_

**c)** El EHR de Lab 1 tenía un campo `subject_id` (entero). Synthea usa un **UUID** como `Id`.  
   ¿Qué ventaja tiene el UUID para sistemas de salud distribuidos?


**Tus respuestas:**

a) ...

b) ...

c) ...


---

## Parte 3 — Análisis demográfico

Antes de analizar enfermedades, entendamos **quiénes** son los pacientes de nuestra población sintética.


In [ ]:
# Calcular edad actual (o edad al momento del fallecimiento)
today = pd.Timestamp(date.today())
patients['CALC_AGE'] = patients.apply(
    lambda r: (r['DEATHDATE'] - r['BIRTHDATE']).days // 365
              if pd.notna(r['DEATHDATE'])
              else (today - r['BIRTHDATE']).days // 365,
    axis=1
)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histograma de edades
axes[0].hist(patients['CALC_AGE'], bins=20,
             color='steelblue', edgecolor='white')
axes[0].set_xlabel('Edad (años)')
axes[0].set_ylabel('Número de pacientes')
axes[0].set_title('Distribución de edades')
axes[0].axvline(patients['CALC_AGE'].mean(), color='red',
                linestyle='--', label=f'Media: {patients["CALC_AGE"].mean():.0f} años')
axes[0].legend()

# Distribución de género
gender_counts = patients['GENDER'].value_counts()
axes[1].bar(gender_counts.index, gender_counts.values,
            color=['steelblue', 'salmon'], edgecolor='white')
axes[1].set_xlabel('Género')
axes[1].set_ylabel('Número de pacientes')
axes[1].set_title('Distribución por género')
for i, (lbl, val) in enumerate(gender_counts.items()):
    axes[1].text(i, val + 0.5, str(val), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f'Total pacientes: {len(patients)}')
print(f'Edad media:      {patients["CALC_AGE"].mean():.1f} años')
print(f'Rango:           {patients["CALC_AGE"].min()}–{patients["CALC_AGE"].max()} años')


In [ ]:
# Distribución de raza y etnicidad
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

race_counts = patients['RACE'].value_counts()
ethnicity_counts = patients['ETHNICITY'].value_counts()

axes[0].barh(race_counts.index, race_counts.values, color='steelblue')
axes[0].set_xlabel('Número de pacientes')
axes[0].set_title('Distribución por raza')

axes[1].barh(ethnicity_counts.index, ethnicity_counts.values, color='teal')
axes[1].set_xlabel('Número de pacientes')
axes[1].set_title('Distribución por etnicidad')

plt.tight_layout()
plt.show()

print('Distribución por raza:')
display(race_counts.to_frame(name='count'))


In [ ]:
# Análisis de mortalidad
n_vivos    = patients['DEATHDATE'].isna().sum()
n_fallecidos = patients['DEATHDATE'].notna().sum()

print(f'Pacientes vivos:      {n_vivos} ({n_vivos/len(patients)*100:.1f}%)')
print(f'Pacientes fallecidos: {n_fallecidos} ({n_fallecidos/len(patients)*100:.1f}%)')

if n_fallecidos > 0:
    fallecidos = patients[patients['DEATHDATE'].notna()].copy()
    fallecidos['EDAD_MUERTE'] = (
        (fallecidos['DEATHDATE'] - fallecidos['BIRTHDATE']).dt.days // 365
    )
    print(f'\nEdad media al fallecimiento: {fallecidos["EDAD_MUERTE"].mean():.1f} años')
    print(f'Rango: {fallecidos["EDAD_MUERTE"].min()}–{fallecidos["EDAD_MUERTE"].max()} años')


#### Ejercicio 3.1 — Interpretar la demografía

**a)** La distribución de raza/etnicidad de esta población sintética refleja los datos del censo de Massachusetts, EE.UU.
   Si quisieras generar una población con demografía guatemalteca, ¿qué limitación tendría usar Synthea directamente?

**b)** ¿Qué dice la distribución de edades sobre el tipo de población modelada?
   ¿Esperarías esta distribución en una clínica pediátrica? ¿Por qué?

**c)** Synthea modela el ciclo completo de vida (nacimiento → muerte).  
   ¿Cómo crees que esto afecta la distribución de condiciones que veremos en la Parte 4?


**Tus respuestas:**

a) ...

b) ...

c) ...


---

## Parte 4 — Análisis epidemiológico

Ahora analizamos las **condiciones clínicas** y **medicamentos** de la población.  
Esto es epidemiología básica: ¿qué enfermedades están presentes y con qué frecuencia?


In [ ]:
# ─── Top 15 condiciones por número de pacientes afectados ─────────────────────
n_total = patients['Id'].nunique()

top_conditions = (
    conditions
    .groupby('DESCRIPTION')
    .agg(n_pacientes=('PATIENT', 'nunique'))
    .reset_index()
    .sort_values('n_pacientes', ascending=False)
    .head(15)
)
top_conditions['prevalencia_pct'] = (
    top_conditions['n_pacientes'] / n_total * 100
).round(1)

# Visualización
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(
    top_conditions['DESCRIPTION'][::-1],
    top_conditions['prevalencia_pct'][::-1],
    color='steelblue'
)
ax.set_xlabel('Prevalencia (%)')
ax.set_title(f'Top 15 condiciones (n = {n_total} pacientes)')
for bar, val in zip(bars, top_conditions['prevalencia_pct'][::-1]):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
            f'{val}%', va='center', fontsize=9)
plt.tight_layout()
plt.show()

print('\nTabla de prevalencias:')
display(top_conditions.reset_index(drop=True))


In [ ]:
# ─── Top 10 medicamentos más frecuentes ───────────────────────────────────────
top_meds = (
    medications
    .groupby('DESCRIPTION')
    .agg(n_prescripciones=('PATIENT', 'count'),
         n_pacientes=('PATIENT', 'nunique'))
    .reset_index()
    .sort_values('n_pacientes', ascending=False)
    .head(10)
)

fig, ax = plt.subplots(figsize=(10, 5))
ax.barh(top_meds['DESCRIPTION'][::-1], top_meds['n_pacientes'][::-1],
        color='teal')
ax.set_xlabel('Número de pacientes')
ax.set_title('Top 10 medicamentos prescritos')
plt.tight_layout()
plt.show()

print('\nNota: los códigos de medicamentos usan RxNorm (como aprendiste en Lab 2)')
print('Ejemplo de códigos:')
display(medications[['DESCRIPTION', 'CODE', 'REASONDESCRIPTION']]
        .drop_duplicates('DESCRIPTION')
        .head(5)
        .reset_index(drop=True))


#### Ejercicio 4.1 — Terminologías en los datos

**a)** En `conditions.csv`, Synthea usa **SNOMED-CT** en lugar de ICD-10.
   ¿Cuál sería la ventaja de usar SNOMED-CT para un simulador de trayectorias clínicas?
   _(Pista: piensa en la granularidad y en los conceptos pre/post-coordinados)_

**b)** Las `observations.csv` usan códigos **LOINC** para laboratorios y signos vitales.
   Busca el código LOINC `8867-4` en https://loinc.org/search/ y describe qué observación representa.

**c)** Compara los top 5 condiciones de esta población con estadísticas epidemiológicas reales
   de Massachusetts. ¿Se parecen? ¿Qué condición esperarías encontrar y no está?


**Tus respuestas:**

a) ...

b) ...

c) ...


---

## Parte 5 — Un Bundle FHIR de Synthea

Además de CSV, Synthea genera un **Bundle FHIR R4** por cada paciente.
Este Bundle es intercambiable directamente con cualquier sistema que soporte FHIR.

Vamos a cargar uno y compararlo con el Bundle que construiste a mano en Lab 3.


In [ ]:
# Cargar el primer Bundle FHIR disponible
fhir_files = sorted(FHIR_DIR.glob('*.json'))

if not fhir_files:
    print('⚠️  No hay archivos FHIR en fhir/')
    print('Copia algunos archivos de synthea/output/fhir/ a labs/lab-3.1-synthetic-data/fhir/')
else:
    fhir_file = fhir_files[0]
    print(f'Cargando: {fhir_file.name}')

    with open(fhir_file, encoding='utf-8') as f:
        raw = json.load(f)

    # Contar recursos por tipo
    tipos = {}
    for entry in raw.get('entry', []):
        rt = entry['resource']['resourceType']
        tipos[rt] = tipos.get(rt, 0) + 1

    print(f'\nBundle con {len(raw["entry"])} recursos:')
    for rt, count in sorted(tipos.items(), key=lambda x: -x[1]):
        print(f'  {count:>4}  {rt}')


In [ ]:
# Parsear con fhir.resources y extraer el recurso Patient
if fhir_files:
    bundle = Bundle.model_validate(raw)

    # Encontrar el recurso Patient
    patient_entry = next(
        e for e in bundle.entry
        if e.resource.resource_type == 'Patient'
    )
    patient_resource = patient_entry.resource

    # Mostrar en formato JSON limpio
    patient_json = json.loads(
        patient_resource.model_dump_json(indent=2, exclude_none=True)
    )
    print(json.dumps(patient_json, indent=2, ensure_ascii=False))


### Comparación: Bundle de Lab 3 vs. Bundle de Synthea

| Característica       | Lab 3 (construido a mano)       | Lab 3.1 (Synthea automático)         |
|----------------------|--------------------------------|--------------------------------------|
| **Escala**           | 1 paciente                     | 100+ pacientes                       |
| **Fuente de datos**  | Base de datos SQL relacional   | Modelo probabilístico epidemiológico |
| **Condiciones**      | ICD-10                         | SNOMED-CT                            |
| **Laboratorios**     | LOINC                          | LOINC                                |
| **Medicamentos**     | —                              | RxNorm                               |
| **FHIR versión**     | R5                             | R4                                   |
| **Tipo de Bundle**   | `transaction` / `collection`   | `transaction`                        |
| **Realismo clínico** | Datos de prueba fijos          | Prevalencias epidemiológicas reales  |
| **Reproducibilidad** | Manual (script SQL)            | Automática (semilla aleatoria)       |

> 💡 Nota que ambos Bundles son FHIR válidos e intercambiables, aunque usen terminologías  
> y versiones ligeramente distintas. Esto es la interoperabilidad en acción.


#### Ejercicio 5.1 — Explorar el Bundle FHIR

Completa el código a continuación para extraer **la primera Condition** del Bundle de Synthea y mostrar:
- El código SNOMED-CT y su descripción
- La referencia al paciente (`subject`)
- La fecha de inicio


In [ ]:
# TODO: Extrae la primera Condition del bundle y muestra sus campos clave
# Pista: itera sobre bundle.entry buscando resource.resource_type == 'Condition'
#        Luego accede a .code.coding[0].code, .code.coding[0].display,
#        .subject.reference y .onset_date_time

if fhir_files:
    # Tu código aquí
    pass


---

## Ejercicios finales

Los siguientes ejercicios consolidan lo aprendido.  
Completa cada uno en orden — los más guiados primero.


### Ejercicio A — Análisis de comorbilidad

Una **comorbilidad** es la presencia simultánea de dos o más condiciones en el mismo paciente.
La combinación diabetes + hipertensión es una de las más frecuentes y clínicamente relevantes.

El código a continuación ya está completo. **Ejecútalo y responde las preguntas al final.**


In [ ]:
# ─── Análisis de comorbilidad: Diabetes + Hipertensión ────────────────────────

# Pacientes con diabetes (cualquier tipo)
diabetes_keywords = ['diabetes', 'Diabetes']
pac_diabetes = set(
    conditions[
        conditions['DESCRIPTION'].str.contains('|'.join(diabetes_keywords), na=False)
    ]['PATIENT']
)

# Pacientes con hipertensión
pac_hta = set(
    conditions[
        conditions['DESCRIPTION'].str.contains('hypertension|Hypertension', na=False)
    ]['PATIENT']
)

# Pacientes con ambas condiciones
pac_ambas = pac_diabetes & pac_hta

print(f'Pacientes con diabetes:        {len(pac_diabetes):>4} ({len(pac_diabetes)/n_total*100:.1f}%)')
print(f'Pacientes con hipertensión:    {len(pac_hta):>4} ({len(pac_hta)/n_total*100:.1f}%)')
print(f'Pacientes con AMBAS (comorbilidad): {len(pac_ambas):>4} ({len(pac_ambas)/n_total*100:.1f}%)')

# ¿Cuál es el riesgo relativo de tener HTA dado que ya tienes DM?
if len(pac_diabetes) > 0:
    prev_hta_en_dm  = len(pac_ambas) / len(pac_diabetes)
    prev_hta_global = len(pac_hta) / n_total
    rr = prev_hta_en_dm / prev_hta_global if prev_hta_global > 0 else float('nan')
    print(f'\nPrevalencia HTA en pacientes con DM:  {prev_hta_en_dm*100:.1f}%')
    print(f'Prevalencia HTA en población total:   {prev_hta_global*100:.1f}%')
    print(f'Riesgo relativo (DM → HTA): {rr:.2f}x')

# Mostrar condiciones adicionales que tienen los pacientes con ambas enfermedades
if pac_ambas:
    otras_condiciones = (
        conditions[
            conditions['PATIENT'].isin(pac_ambas) &
            ~conditions['DESCRIPTION'].str.contains('diabetes|Diabetes|hypertension|Hypertension', na=False)
        ]
        .groupby('DESCRIPTION')
        .agg(n=('PATIENT', 'nunique'))
        .sort_values('n', ascending=False)
        .head(5)
    )
    print('\nOtras condiciones frecuentes en pacientes con DM + HTA:')
    display(otras_condiciones)


### Ejercicio B — Condiciones por grupo etario

Las enfermedades no se distribuyen uniformemente por edad. Un análisis estratificado por grupos etarios
revela qué condiciones afectan a qué grupos.

**Instrucciones:**

1. Crea grupos etarios: `[0-17]`, `[18-44]`, `[45-64]`, `[65+]`
2. Para cada grupo, encuentra las **3 condiciones más frecuentes** (por número de pacientes únicos)
3. Muestra los resultados en una tabla o gráfica

**Pistas:**
- Usa `pd.cut()` con `bins=[0, 18, 45, 65, 200]`
- Merge `conditions` con `patients` para obtener la edad de cada paciente al diagnóstico
- Usa `.groupby()` con múltiples columnas


In [ ]:
# Paso 1: Merge conditions con patients para obtener BIRTHDATE
conds_con_edad = conditions.merge(
    patients[['Id', 'BIRTHDATE']],
    left_on='PATIENT',
    right_on='Id',
    how='left'
)

# Paso 2: Calcular edad aproximada al momento del diagnóstico
conds_con_edad['edad_dx'] = (
    (conds_con_edad['START'] - conds_con_edad['BIRTHDATE']).dt.days // 365
).clip(lower=0)

# Paso 3: Crear grupos etarios
# TODO: Define los grupos etarios usando pd.cut()
# bins=[0, 18, 45, 65, 200], labels=['0-17', '18-44', '45-64', '65+']
conds_con_edad['grupo_etario'] = ...  # TODO

# Paso 4: Top 3 condiciones por grupo etario
# TODO: Agrupa por grupo_etario y DESCRIPTION, cuenta pacientes únicos,
#       luego toma el top 3 de cada grupo
top_por_grupo = ...  # TODO

# Paso 5: Mostrar resultados
# TODO: Muestra la tabla o crea una visualización
display(top_por_grupo)


### Ejercicio C — Explorar un módulo de Synthea

Cada enfermedad en Synthea está modelada como un **módulo JSON** que define
una máquina de estados probabilística. Vamos a leer el módulo de hipertensión.

**Instrucciones:**

1. Ejecuta la celda de carga (está escrita para ti)
2. Responde las preguntas con base en el módulo cargado


In [ ]:
# Intentar cargar el módulo de hipertensión de Synthea
# Ajusta SYNTHEA_DIR si Synthea está en una ubicación diferente
import os

SYNTHEA_DIR = Path('../../../../synthea')  # Ruta relativa desde el notebook
module_path = SYNTHEA_DIR / 'src/main/resources/modules/hypertension.json'

if not module_path.exists():
    print(f'⚠️  No se encontró el módulo en: {module_path.resolve()}')
    print('Ajusta SYNTHEA_DIR en la celda anterior.')
else:
    with open(module_path, encoding='utf-8') as f:
        module = json.load(f)

    print(f'Módulo: {module["name"]}')
    print(f'Total de estados: {len(module["states"])}')
    print()
    print('Estados y sus tipos:')
    print(f'{"Estado":45} {"Tipo"}')
    print('-' * 70)
    for state_name, state_data in module['states'].items():
        print(f'{state_name:45} {state_data["type"]}')


#### Preguntas sobre el módulo de hipertensión

Usando la salida del módulo (o leyendo el JSON directamente):

**a)** ¿Cuántos estados tiene el módulo? ¿Cuál es el estado inicial?

**b)** Encuentra el estado `Diagnose_Hypertension` en el JSON.  
   ¿Qué código SNOMED-CT se asigna al diagnóstico de hipertensión?
   Busca ese código en https://browser.ihtsdotools.org/ y confirma su descripción.

**c)** ¿Cómo modela el módulo la transición desde "sin tratamiento" hasta "con medicamento"?
   Describe el flujo con nombres de estados específicos del módulo.


**Tus respuestas:**

a) ...

b) ...

c) ...


### Ejercicio D — Reflexión final

Responde las siguientes preguntas en 3–5 oraciones cada una.
No hay respuesta única correcta — se evalúa el razonamiento.

**1.** Los datos que analizaste en este lab fueron generados a partir de estadísticas de Massachusetts, EE.UU.
   Si usaras estos datos sintéticos para entrenar un modelo de predicción de riesgo cardiovascular
   para una clínica en Guatemala, ¿qué problemas podrían surgir?
   ¿Cómo mitigarías ese riesgo?

**2.** En Lab 3 construiste un Bundle FHIR a mano usando datos de un EHR relacional.
   En este lab viste cómo Synthea genera Bundles FHIR automáticamente.
   ¿En qué escenario preferirías el enfoque manual (Lab 3) y en cuál el automático (Synthea)?

**3.** ¿Qué tipo de información clínicamente importante **no puede capturar** Synthea, aunque lo intente?
   Piensa en aspectos como experiencia subjetiva del paciente, contexto social, variaciones locales, etc.

**4.** El gobierno de Guatemala quisiera usar datos sintéticos para planificar la distribución
   de recursos de salud (médicos, medicamentos, camas hospitalarias) a nivel departamental.
   ¿Sería apropiado usar Synthea directamente? ¿Qué alternativa propondrías?


**Tus respuestas:**

1. ...

2. ...

3. ...

4. ...
